In [1]:
# %cd "/content/drive/MyDrive/My Projects/face-verification-with-siamese-network"

In [2]:
# !pip install import-ipynb

In [3]:
# import import_ipynb
# import config

from keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def make_pairs(images , labels):
  # initialize the pairImages and pairLabels list so we can save 
  # the pair images in former list and save the corresponding label
  # into the latter list
  pairImages = []
  pairLabels = []

  # get the number of unique calsses in the dataset
  numClasses = len(np.unique(labels))
  
  # build a list that represents the indeces of each label 
  # in the dataset
  idx = [np.where(labels == i)[0] for i in range(0 , numClasses)]

  # loop over all the images
  for idxA in range(len(images)):
    # grab the corresponding image and label
    currentImage = images[idxA]
    label = labels[idxA]

    # pick an image that has the same label as
    # privious image
    idxB = np.random.choice(idx[label])
    posImage = images[idxB]

    pairImages.append([currentImage, posImage])
    pairLabels.append([1])

    # select specific indeces that have a different label as the 
    # current label. Then select a random index from these specific
    # indeces and grab the corresponding image.
    negIdx = np.where(labels != label)[0]
    negImage = images[np.random.choice(negIdx)]

    pairImages.append([currentImage, negImage])
    pairLabels.append([0])

  return (np.array(pairImages), np.array(pairLabels))

In [5]:
def euclidean_distance(vectors):
  (featsA, featsB) = vectors

  sumSquared = K.sum(K.square(featsA - featsB) , axis = 1 , keepdims=True)

  return K.sqrt(K.maximum(sumSquared , K.epsilon()))

In [6]:
def plot_training(H, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.title("Contrastive Loss")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

In [7]:
def contrastive_loss(y, preds , margin=1):
  # cast the ground truth value dtype into prediction
  # value datatype to have the same datatype. in order to
  # able to do some calculations between y and preds
  y = tf.cast(y , preds.dtype)

  # calcualte the contrastive loss function via
  # tensorflow backend API
  squaredPreds = K.square(preds)
  squaredMargin = K.square(K.maximum(margin - preds , 0))
  loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

  # return the computed contrastive loss
  return loss

In [8]:
class learning_rate_schedule():
  def plot(self, epochs, title="Learning Rate Schedule"):
    lrs = [self(i) for i in epochs]
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(epochs, lrs)
    plt.title(title)
    plt.xlabel("Epoch #")
    plt.ylabel("Learning Rate")
    plt.show()

class step_decay(learning_rate_schedule):
  def __init__(self, initAlpha = 0.001 , factor = 0.25 , dropEvery = 10):
    self.initAlpha = initAlpha
    self.factor = factor
    self.dropEvery = dropEvery

  def __call__(self, epoch):
    # compute the learning rate for the current epoch
    exp = np.floor((1 + epoch) / self.dropEvery)
    alpha = self.initAlpha * (self.factor ** exp)
    # return the learning rate
    return float(alpha)

class polynomial_decay(learning_rate_schedule):
	def __init__(self, maxEpochs=20, initAlpha=0.001, power=1.0):
		# store the maximum number of epochs, base learning rate,
		# and power of the polynomial
		self.maxEpochs = maxEpochs
		self.initAlpha = initAlpha
		self.power = power
	def __call__(self, epoch):
		# compute the new learning rate based on polynomial decay
		decay = (1 - (epoch / float(self.maxEpochs))) ** self.power
		alpha = self.initAlpha * decay
		# return the new learning rate
		return float(alpha)

In [9]:
schedule = "step_decay"

if schedule == "linear":
  schedule = polynomial_decay(maxEpochs = 100 , initAlpha = 1e-3 , power = 1.0)

if schedule == "polynomial":
  schedule = polynomial_decay(maxEpochs = 100 , initAlpha =1e-3 , power = 3.0)

if schedule == "step_decay":
  schedule = step_decay(initAlpha = 1e-3 , factor = 0.25 , dropEvery = 20)

lrate = LearningRateScheduler(schedule)